In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

# get dynarev data

In [ ]:
# fil_path = [
#     f
#     for f in fs.glob(
#         f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={year}/skjema=RA-0174-1/*"
#     )
#     if f.endswith(".parquet")
# ]

# # Use the ParquetDataset to read multiple files
# dataset = pq.ParquetDataset(fil_path, filesystem=fs)
# table = dataset.read()

# # Convert to Pandas DataFrame
# skjema = table.to_pandas()

# # Now you can work with the DataFrame `skjema`
# skjema

In [ ]:
felt_id_values = [
    "V_ORGNR",
    "F_ADRESSE",
    "FJOR_NACE_B_T1",
    "TMP_SN2007_5",
    "B_KOMMUNENR",
    "REGTYPE",
    "B_SYSSELSETTING_SYSS",
    "TMP_NY_BDR_SYSS",
    "GJELDENDE_BDR_SYSS",
    "FJOR_SYSSEL_T1",
    "LONN_PST_AORDN",
    "GJELDENDE_LONN_KR",
    "LONN",
    "FJOR_LONN_KR_T1",
    "TMP_SNITTLONN",
    "FJOR_SNITTLONN_T1",
    "GJELDENDE_OMSETN_KR",
    "OMSETN_KR",
    "FJOR_OMSETM_KR_T1",
    "TMP_SNITTOMS",
    "FJOR_SNITTOMS_T1",
    "TMP_SALGSINT_BED",
    "TMP_FORBRUK_BED",
    "VAREKOST_BED",
    "GJELDENDE_DRIFTSK_KR",
    "DRIFTSKOST_KR",
    "FJOR_DRIFTSKOST_KR_T1",
    "NACEF_5",
    "SALGSINT",
    "FORBRUK",
    "TMP_NO_P4005",
    "TMP_AVPROS_ORGFORB",
    "ORGNR_N_1",
    "TMP_NO_OMSETN",
    "TMP_DRIFTSKOSTNAD_9010",
    "TMP_DRIFTSKOSTNAD_9910",
]

# Assuming `skjema` is your DataFrame and `felt_id_values` is your list of values
skjema = skjema[skjema["feltnavn"].isin(felt_id_values)]

skjema

In [ ]:
pivot_df = skjema.pivot_table(
    index=["id", "radnr"], columns="feltnavn", values="feltverdi", aggfunc="first"
)
pivot_df = pivot_df.reset_index()
pivot_df.columns = pivot_df.columns.str.lower()

In [ ]:
# Create the boolean mask where either condition is true
mask = pivot_df["id"] == enhets_id

# Apply the mask to filter the DataFrame
bedrift_liste = pivot_df[mask]

# Fill NaN values with 0 for driftskostnad columns in the DataFrame
bedrift_liste["tmp_driftskostnad_9010"].fillna(0, inplace=True)
bedrift_liste["tmp_driftskostnad_9910"].fillna(0, inplace=True)

# Extract values from the row where `rad_nr` is 0
values_to_assign = bedrift_liste[bedrift_liste["radnr"] == 0].iloc[0]

# Convert the driftskostnad values to numeric types, in case they are not already
driftskostnad_9010_value = float(values_to_assign["tmp_driftskostnad_9010"])
driftskostnad_9910_value = float(values_to_assign["tmp_driftskostnad_9910"])

# Determine the greater value between tmp_driftskostnad_9010 and tmp_driftskostnad_9910 for the row where `rad_nr` is 0
drkost_total_value = max(driftskostnad_9010_value, driftskostnad_9910_value)

# Assign these values to new columns in the entire DataFrame
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "orgnr_foretak"] = values_to_assign[
    "orgnr_n_1"
]
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "naring_f"] = values_to_assign["nacef_5"]
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "forbruk_total"] = values_to_assign[
    "forbruk"
]
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "salgsint_total"] = values_to_assign[
    "salgsint"
]
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "omsetn_total"] = values_to_assign[
    "tmp_no_omsetn"
]
bedrift_liste.loc[bedrift_liste["radnr"] != 0, "drkost_total"] = drkost_total_value
bedrift_liste.loc[
    bedrift_liste["radnr"] != 0, "varekostnader_total"
] = values_to_assign["tmp_no_p4005"]

# Remove the rows where `rad_nr` is 0
bedrift_liste = bedrift_liste[bedrift_liste["radnr"] != 0]

# Optional: Reset the index after deletion
bedrift_liste.reset_index(drop=True, inplace=True)

bedrift_liste = bedrift_liste[
    [
        "orgnr_bedrift",
        "omsetning",
        "sysselsetting_syss",
        "naring4",
        "naring_f_4",
        "naring3",
        "naring_f_3",
        "naring_f",
        "forbruk_total",
        "salgsint_total",
        "tmp_no_omsetn",
        "omsetn_total",
        "drkost_total",
    ]
]